## Preprocessing

In [52]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv
application_df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [53]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#app_df_clean = application_df.drop(columns=['Race'])
app_df_clean.head(2)

,BMI,PhysicalHealth,MentalHealth,SleepTime,HeartDisease_No,HeartDisease_Yes,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,...,GenHealth_Fair,GenHealth_Good,GenHealth_Poor,GenHealth_Very good,Asthma_No,Asthma_Yes,KidneyDisease_No,KidneyDisease_Yes,SkinCancer_No,SkinCancer_Yes
0,16.60,3.0,30.0,5.0,1,0,0,1,1,0,...,0,0,0,1,0,1,1,0,0,1
1,20.34,0.0,0.0,7.0,1,0,1,0,1,0,...,0,0,0,1,1,0,1,0,1,0


In [54]:
# Convert categorical data to numeric with `pd.get_dummies`
app_df_clean = pd.get_dummies(app_df_clean)
app_df_clean.head()

,BMI,PhysicalHealth,MentalHealth,SleepTime,HeartDisease_No,HeartDisease_Yes,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,...,GenHealth_Fair,GenHealth_Good,GenHealth_Poor,GenHealth_Very good,Asthma_No,Asthma_Yes,KidneyDisease_No,KidneyDisease_Yes,SkinCancer_No,SkinCancer_Yes
0,16.60,3.0,30.0,5.0,1,0,0,1,1,0,...,0,0,0,1,0,1,1,0,0,1
1,20.34,0.0,0.0,7.0,1,0,1,0,1,0,...,0,0,0,1,1,0,1,0,1,0
2,26.58,20.0,30.0,8.0,1,0,0,1,1,0,...,1,0,0,0,0,1,1,0,1,0
3,24.21,0.0,0.0,6.0,1,0,1,0,1,0,...,0,1,0,0,1,0,1,0,0,1
4,23.71,28.0,0.0,8.0,1,0,1,0,1,0,...,0,0,0,1,1,0,1,0,1,0


In [55]:
# Assuming your dataset is stored in a variable named 'app_df_clean'
target_column = 'HeartDisease_Yes'  # Replace 'HeartDisease_Yes' with the appropriate column name containing the class labels

target = app_df_clean.loc[:, target_column]  # Access the target column using the 'loc' method

# Convert the target attribute to the desired data type if needed
target = target.astype(int)  # Convert to integer type if the class labels are stored as strings

# Print the target attribute
print(target)


0         0
1         0
2         0
3         0
4         0
         ..
319790    1
319791    0
319792    0
319793    0
319794    0
Name: HeartDisease_Yes, Length: 319795, dtype: int32


In [56]:
# Determine the number of unique values in each column.
app_df_clean.nunique()

BMI                                 3604
PhysicalHealth                        31
MentalHealth                          31
SleepTime                             24
HeartDisease_No                        2
HeartDisease_Yes                       2
Smoking_No                             2
Smoking_Yes                            2
AlcoholDrinking_No                     2
AlcoholDrinking_Yes                    2
Stroke_No                              2
Stroke_Yes                             2
DiffWalking_No                         2
DiffWalking_Yes                        2
Sex_Female                             2
Sex_Male                               2
AgeCategory_18-24                      2
AgeCategory_25-29                      2
AgeCategory_30-34                      2
AgeCategory_35-39                      2
AgeCategory_40-44                      2
AgeCategory_45-49                      2
AgeCategory_50-54                      2
AgeCategory_55-59                      2
AgeCategory_60-6

In [57]:
# Look at PhysicalHealth value counts for binning
application_counts = app_df_clean['PhysicalHealth'].value_counts()
application_counts

0.0     226589
30.0     19509
2.0      14880
1.0      10489
3.0       8617
5.0       7606
10.0      5453
15.0      5012
7.0       4629
4.0       4468
20.0      3216
14.0      2893
6.0       1270
25.0      1164
8.0        924
21.0       626
12.0       605
28.0       446
29.0       204
9.0        180
18.0       167
16.0       135
27.0       124
17.0       110
13.0        91
22.0        89
11.0        85
24.0        67
26.0        66
23.0        46
19.0        35
Name: PhysicalHealth, dtype: int64

In [58]:
# Look at SleepTime value counts for binning
application_counts = app_df_clean['SleepTime'].value_counts()
application_counts

7.0     97751
8.0     97602
6.0     66721
5.0     19184
9.0     16041
10.0     7796
4.0      7750
12.0     2205
3.0      1992
2.0       788
1.0       551
11.0      415
14.0      243
16.0      236
15.0      189
18.0      102
13.0       97
20.0       64
24.0       30
17.0       21
22.0        9
19.0        3
23.0        3
21.0        2
Name: SleepTime, dtype: int64

In [59]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts > 21].index)


# Replace in dataframe
for app in application_types_to_replace:
    application_df['SleepTime'] = application_df['SleepTime'].replace(app,"Other")

# Check to make sure binning was successful
application_df['SleepTime'].value_counts()

Other    319757
17.0         21
22.0          9
19.0          3
23.0          3
21.0          2
Name: SleepTime, dtype: int64

In [60]:
# Convert categorical data to numeric with `pd.get_dummies`
app_df_clean = pd.get_dummies(app_df_clean)
app_df_clean.head()

,BMI,PhysicalHealth,MentalHealth,SleepTime,HeartDisease_No,HeartDisease_Yes,Smoking_No,Smoking_Yes,AlcoholDrinking_No,AlcoholDrinking_Yes,...,GenHealth_Fair,GenHealth_Good,GenHealth_Poor,GenHealth_Very good,Asthma_No,Asthma_Yes,KidneyDisease_No,KidneyDisease_Yes,SkinCancer_No,SkinCancer_Yes
0,16.60,3.0,30.0,5.0,1,0,0,1,1,0,...,0,0,0,1,0,1,1,0,0,1
1,20.34,0.0,0.0,7.0,1,0,1,0,1,0,...,0,0,0,1,1,0,1,0,1,0
2,26.58,20.0,30.0,8.0,1,0,0,1,1,0,...,1,0,0,0,0,1,1,0,1,0
3,24.21,0.0,0.0,6.0,1,0,1,0,1,0,...,0,1,0,0,1,0,1,0,0,1
4,23.71,28.0,0.0,8.0,1,0,1,0,1,0,...,0,0,0,1,1,0,1,0,1,0


In [61]:
# Split our preprocessed data into our features and target arrays
y = app_df_clean['HeartDisease_Yes'].values
X = app_df_clean.drop(columns=['HeartDisease_Yes']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [62]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_features = len(X_train[0])


nn_model = tf.keras.models.Sequential()

nn = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu", input_dim=num_input_features))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu", input_dim=50))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()




Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 12)                552       
                                                                 
 dense_11 (Dense)            (None, 12)                156       
                                                                 
 dense_12 (Dense)            (None, 1)                 13        
                                                                 
Total params: 721
Trainable params: 721
Non-trainable params: 0
_________________________________________________________________


In [67]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [68]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=25)


Epoch 1/25


C:\Users\vince\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


7496/7496 [==============================] - 9s 1ms/step - loss: 0.0000e+00 - accuracy: 0.9178
Epoch 2/25
7496/7496 [==============================] - 9s 1ms/step - loss: 0.0000e+00 - accuracy: 0.9144
Epoch 3/25
7496/7496 [==============================] - 8s 1ms/step - loss: 0.0000e+00 - accuracy: 0.9144
Epoch 4/25
7496/7496 [==============================] - 8s 1ms/step - loss: 0.0000e+00 - accuracy: 0.9144
Epoch 5/25
7496/7496 [==============================] - 8s 1ms/step - loss: 0.0000e+00 - accuracy: 0.9144
Epoch 6/25
7496/7496 [==============================] - 7s 935us/step - loss: 0.0000e+00 - accuracy: 0.9144
Epoch 7/25
7496/7496 [==============================] - 7s 940us/step - loss: 0.0000e+00 - accuracy: 0.9144
Epoch 8/25
7496/7496 [==============================] - 7s 955us/step - loss: 0.0000e+00 - accuracy: 0.9144
Epoch 9/25
7496/7496 [==============================] - 7s 940us/step - loss: 0.0000e+00 - accuracy: 0.9144
Epoch 10/25
7496/7496 [==========================

In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2499/2499 - 2s - loss: 0.0000e+00 - accuracy: 0.9144 - 2s/epoch - 649us/step
Loss: 0.0, Accuracy: 0.9144079089164734
